Import Data
- Make a dataframe from the csv
    - There is some inconsistent data (some rows formatted poorly), so we will skip them    


In [9]:
import pandas as pd

FILE = "./data/Assignment-1_Data.csv"

data = pd.read_csv(FILE, sep=";", on_bad_lines="skip", low_memory=False)

print(data)

        BillNo                             Itemname  Quantity  \
0       536365   WHITE HANGING HEART T-LIGHT HOLDER         6   
1       536365                  WHITE METAL LANTERN         6   
2       536365       CREAM CUPID HEARTS COAT HANGER         8   
3       536365  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4       536365       RED WOOLLY HOTTIE WHITE HEART.         6   
...        ...                                  ...       ...   
522059  581587          PACK OF 20 SPACEBOY NAPKINS        12   
522060  581587          CHILDREN'S APRON DOLLY GIRL         6   
522061  581587         CHILDRENS CUTLERY DOLLY GIRL         4   
522062  581587      CHILDRENS CUTLERY CIRCUS PARADE         4   
522063  581587         BAKING SET 9 PIECE RETROSPOT         3   

                    Date Price  CustomerID         Country  
0       01.12.2010 08:26  2,55     17850.0  United Kingdom  
1       01.12.2010 08:26  3,39     17850.0  United Kingdom  
2       01.12.2010 08:26  2,75     1

Clean Data
- Only data we care about is BillNo, Itemname, and Country 
    - We drop the other columns
- Remove entries that are missing data (BillNo, Itemname, or Country)
 

In [10]:
columns_to_keep = ['BillNo', 'Itemname', 'Country']

data = data[columns_to_keep]

#Drop rows with missing values
data.dropna(inplace=True)

print(data)

        BillNo                             Itemname         Country
0       536365   WHITE HANGING HEART T-LIGHT HOLDER  United Kingdom
1       536365                  WHITE METAL LANTERN  United Kingdom
2       536365       CREAM CUPID HEARTS COAT HANGER  United Kingdom
3       536365  KNITTED UNION FLAG HOT WATER BOTTLE  United Kingdom
4       536365       RED WOOLLY HOTTIE WHITE HEART.  United Kingdom
...        ...                                  ...             ...
522059  581587          PACK OF 20 SPACEBOY NAPKINS          France
522060  581587          CHILDREN'S APRON DOLLY GIRL          France
522061  581587         CHILDRENS CUTLERY DOLLY GIRL          France
522062  581587      CHILDRENS CUTLERY CIRCUS PARADE          France
522063  581587         BAKING SET 9 PIECE RETROSPOT          France

[520609 rows x 3 columns]


Group Data
- Group transaction data by country
- Find countries with a decent number of rows (transaction data)  
- We keep the countries with a lot of transaction data 
        - United Kingdom, France, and Germany  

In [11]:
country_datas = {country: data for country, data in data.groupby('Country')}

for country, data in country_datas.items():
        print(f"Number of Rows for {country}:")
        print(data.shape[0]) # number of rows
        print("\n")
        
countries_to_keep = ['United Kingdom', 'France', 'Germany']

country_datas = {key: value for key, value in country_datas.items() if key in countries_to_keep}

for country, df in country_datas.items():
        print(f"Data for {country}:")
        print(df) # number of rows
        print("\n")

Data for Australia:
1185


Data for Austria:
398


Data for Bahrain:
18


Data for Belgium:
2031


Data for Brazil:
32


Data for France:
8408


Data for Germany:
9042


Data for Greece:
145


Data for Hong Kong:
284


Data for Iceland:
182


Data for Israel:
295


Data for Italy:
758


Data for Japan:
321


Data for Lebanon:
45


Data for Lithuania:
35


Data for Malta:
112


Data for Netherlands:
2363


Data for Norway:
1072


Data for Poland:
330


Data for Portugal:
1501


Data for RSA:
58


Data for Saudi Arabia:
9


Data for Singapore:
222


Data for Spain:
2485


Data for Sweden:
451


Data for Switzerland:
1967


Data for USA:
179


Data for United Arab Emirates:
68


Data for United Kingdom:
486167


Data for Unspecified:
446


Data for France:
        BillNo                         Itemname Country
26      536370        ALARM CLOCK BAKELIKE PINK  France
27      536370         ALARM CLOCK BAKELIKE RED  France
28      536370       ALARM CLOCK BAKELIKE GREEN  France
29      5363